In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Mengatur jumlah lahan parkir
total_parking_spots = 134
max_parking_spots = 134  # Menentukan batas maksimum lahan parkir

In [ ]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Mengubah ukuran gambar
    img = img / 255.0  # Normalisasi
    return img

In [ ]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # 1 neuron untuk binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
dataset_path = '/content/drive/MyDrive/cars/train' #ganti path

In [ ]:
if not os.path.exists(dataset_path):
    print(f"Dataset tidak ditemukan di {dataset_path}")
else:
    print(f"Dataset ditemukan di {dataset_path}")

Dataset ditemukan di /content/drive/MyDrive/cars/train


In [ ]:
dataset_path = os.path.abspath('/content/drive/MyDrive/cars/train')  # Ganti path
print("Jalur dataset absolut:", dataset_path)

Jalur dataset absolut: /content/drive/MyDrive/cars/train


In [ ]:
print("Isi folder train:", os.listdir(dataset_path))
if 'mobil' in os.listdir(dataset_path):
    print("Isi folder mobil:", os.listdir(os.path.join(dataset_path, 'mobil')))
if 'bukan_mobil' in os.listdir(dataset_path):
    print("Isi folder bukan mobil:", os.listdir(os.path.join(dataset_path, 'bukan_mobil')))

Isi folder train: ['mobil', 'bukan mobil']
Isi folder mobil: ['images7.jpg', 'images9.jpg', 'images2.jpg', 'images6.jpg', 'images10.jpg', 'images1.jpg', 'images4.jpg', 'images3.jpg', 'images8.jpg', 'images5.jpg']


In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/cars/train',  # Ganti path
    target_size=(224, 224),
    batch_size=34,
    class_mode='binary'
)


Found 20 images belonging to 2 classes.


In [ ]:
model = create_model()
model.fit(train_generator, epochs=10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.5000 - loss: 0.6846
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.6500 - loss: 0.8138
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5000 - loss: 22.0569
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 9.9145
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 1.0000 - loss: 0.1839
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.8000 - loss: 0.4660
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 1.0000 - loss: 0.0045
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 1.0000 - loss: 0.0085
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.5000 - loss: 0.9499
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 1.0000 - loss: 6.3079e-04


In [ ]:
def classify_parking(image_path):
    global total_parking_spots
    image = load_and_preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image)
     # Jika prediksi lebih besar dari 0.5, maka itu mobil
    if prediction[0][0] > 0.5:
        total_parking_spots -= 1
        print("Ini adalah mobil MASUK. Jumlah lahan parkir tersisa:", total_parking_spots)
    else:
        print("Ini bukan mobil. Jumlah lahan parkir tersisa:", total_parking_spots)

# Contoh penggunaan
classify_parking('/content/drive/MyDrive/parkir mobil 4.jpg')  # Ganti dengan path ke gambar yang ingin diuji

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Ini bukan mobil. Jumlah lahan parkir tersisa: 134


In [ ]:
def car_exited():
    global total_parking_spots
    if total_parking_spots < max_parking_spots:
        total_parking_spots += 1
        print("Mobil sudah keluar. Jumlah lahan parkir sekarang:", total_parking_spots)
    else:
        print("Jumlah lahan parkir sudah maksimum, tidak bisa menambah.")

classify_parking('/content/drive/MyDrive/parkir mobil 4.jpg')  # Ganti dengan path ke gambar yang ingin diuji
# Setelah mobil keluar, panggil fungsi car_exited
car_exited()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Ini bukan mobil. Jumlah lahan parkir tersisa: 134
Jumlah lahan parkir sudah maksimum, tidak bisa menambah.


In [ ]:
def classify_parking(image_path):
    global total_parking_spots
    image = load_and_preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image)
     # Jika prediksi lebih besar dari 0.5, maka itu mobil
    if prediction[0][0] > 0.5:
        total_parking_spots -= 1
        print("Ini adalah mobil MASUK. Jumlah lahan parkir tersisa:", total_parking_spots)
    else:
        print("Ini bukan mobil. Jumlah lahan parkir tersisa:", total_parking_spots)
# Contoh penggunaan
classify_parking('/content/drive/MyDrive/parkir mobil 3.jpg')  # Ganti dengan path ke gambar yang ingin diuji

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Ini adalah mobil MASUK. Jumlah lahan parkir tersisa: 133


In [ ]:
def car_exited():
    global total_parking_spots
    if total_parking_spots < max_parking_spots:
        total_parking_spots += 1
        print("Mobil sudah keluar. Jumlah lahan parkir sekarang:", total_parking_spots)
    else:
        print("Jumlah lahan parkir sudah maksimum, tidak bisa menambah.")

classify_parking('/content/drive/MyDrive/parkir mobil 2.jpg')  # Ganti dengan path ke gambar yang ingin diuji
# Setelah mobil keluar, panggil fungsi car_exited
car_exited()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Ini bukan mobil. Jumlah lahan parkir tersisa: 133
Mobil sudah keluar. Jumlah lahan parkir sekarang: 134
